In [ ]:
import argparse
import multiprocessing
import os
import time
from collections import defaultdict

import numpy as np
import yaml

from config import SPECS_COMPLETE_YAMLS, EV_RESULTS_DIR, EEV_RESULTS_DIR
from model import DC, LPAC, Solution
from util import in_notebook

In [ ]:
# user inputs
if in_notebook():
    f = 0
    r_hat = 3
    pftype = 'dc'
    casestudy = 'imelda'
    approach = 'stochastic'
    timelimit = 10 * 60
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--f')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype')
    parser.add_argument('--approach')
    parser.add_argument('--timelimit')
    args = parser.parse_args()
    f = int(args.f)
    r_hat = int(args.rhat)
    pftype = str(args.pftype)
    casestudy = str(args.casestudy)
    approach = str(args.approach)
    timelimit = int(args.timelimit)

In [ ]:
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype]) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    specs['options']['approach'] = approach
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_hat)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]

In [ ]:
model_classes = {
    'dc': DC,
    'lpacc': LPAC,
    'lpacf': LPAC,
    'qpac': LPAC
}

In [ ]:
# get the appropriate model class
modelcls = model_classes[pftype]

# instantiate a model
MODEL = modelcls(**specs)

# budget constraint
MODEL.con_resource_hi.RHS = f

# update
MODEL.update()

Below, it is necessary to fix the variable to the _rounded_ value produced by the EV problem. This is because those values can be 1e-6 (or 1 - 1e-6) which induces infeasibility in certain constraints.

In [ ]:
# fix the EV solution
ev_solution_filename = os.path.join(EV_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
solution = Solution.from_zip(ev_solution_filename)
for (k, r), value in solution['x'].items():
    if value == 1:
        print(k, r)
    MODEL.x[k, r].lb = MODEL.x[k, r].ub = int(value)

In [ ]:
MODEL.model.setParam('MIPGap', 0.00)
MODEL.model.setParam('TimeLimit', timelimit)
MODEL.model.setParam('Threads', np.floor(multiprocessing.cpu_count()))
MODEL.model.setParam('ImpliedCuts', 2)
MODEL.model.setParam('PreSolve', 2)
MODEL.model.setParam('CutPasses', 10)
MODEL.model.setParam('MIPFocus', 2)
MODEL.update()
MODEL.solve()

os.makedirs(EEV_RESULTS_DIR, exist_ok=True)
eev_solution_filename = os.path.join(EEV_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
sol = Solution.from_solved_instance(MODEL)
sol.to_zip(eev_solution_filename, variables=['x', 'ObjBound', 'ObjVal', 'gamma', 'gamma_under', 'gamma_over'])